<a href="https://colab.research.google.com/github/Hirunikathaluwage/RAG/blob/main/Simple_RAG_Application_using_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain -qU
!pip install langchain-openai -qU
!pip install langchain-chroma -qU
!pip install langchain chromadb

In [ ]:
#import necessary libraries
import os
from google.colab import userdata

#initialize language model

In [ ]:
from langchain_openai import ChatOpenAI
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

#initialize the ChatOpenAI model
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)


#initialize Embedding Model

In [ ]:
from langchain_openai import OpenAIEmbeddings
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

In [ ]:
!pip install -U langchain-community -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00


#Create embedded documents


In [ ]:
from langchain_core.documents import Document
documents = [
    Document(
        page_content=" New app nexora 2025 has been launched.",
        METADATA ={"source" "mobile app"},),

    Document(
        page_content=" New desktop app new 2025 has been launched.",
        METADATA ={"source" "desktop app"},),
    Document(
        page_content=" The AI revolution continous to transform industries and reshape the global economy.Significant advancments in articals",
        METADATA ={"source" "ai revolution news"},),
]

In [ ]:
from langchain_chroma import Chroma
vectorstore = Chroma.from_documents(documents, embedding=embedding_model
)

#Perform Similarity Search

In [ ]:
results = vectorstore.similarity_search("mobile app")

for results in results:
  print(results.page_content)
  print(results.metadata)



 New app nexora 2025 has been launched.
{}
 New desktop app new 2025 has been launched.
{}
 The AI revolution continus to transform industries and reshape the global economy.Significant advancments in articals
{}
